In [1]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()


Using TensorFlow backend.


In [30]:
texts = []

with open('ALL_reviews.txt', encoding='utf-8') as f:
    for line in f:
        texts.append(line)
        
with open('all_my_reviews.txt', encoding='utf-8') as f:
    for line in f:
        texts.append(line)       
        
        
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

max_words = 10000 # Max size of the dictionary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)


# Flatten the list of lists resulting from the tokenization. This will reduce the list
# to one dimension, allowing us to apply the sliding window technique to predict the next word
text = [item for sublist in sequences for item in sublist]

vocab_size = len(tokenizer.word_index)

In [3]:
vocab_size

45545

In [4]:
sentence_len = 15

pred_len = 5

train_len = sentence_len - pred_len

seq = []

# Sliding window to generate train data
for i in range(len(text)-sentence_len):
    seq.append(text[i:i+sentence_len])

# Reverse dictionary to decode tokenized sequences back to words
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

In [35]:
# Each row in seq is a 15 word long window. We append he first 10 words as the input to predict the next word
trainX = []
trainy = []

for i in seq:
    trainX.append(i[:train_len])
    trainy.append(i[-1])

In [6]:
reverse_word_map[1514]

'реплики'

In [38]:
#define model

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout


model = Sequential([
    Embedding(max_words, 50, input_length=train_len),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(max_words-1, activation='softmax')
])

In [119]:
# Train model with checkpoints
from keras.callbacks import ModelCheckpoint
import numpy
import pandas as pd


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



filepath = "./model_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
history = model.fit(numpy.asarray(trainX),
         pd.get_dummies(numpy.asarray(trainy)),
         epochs = 50,
         batch_size = 128,
         callbacks = callbacks_list,
         verbose = 1)

Epoch 1/50
234366/234366 [==============================] - 129s 550us/step - loss: 7.2311 - accuracy: 0.0463

Epoch 00001: loss improved from inf to 7.23112, saving model to ./model_weights.hdf5
Epoch 2/50
234366/234366 [==============================] - 124s 529us/step - loss: 7.1557 - accuracy: 0.0462

Epoch 00002: loss improved from 7.23112 to 7.15572, saving model to ./model_weights.hdf5
Epoch 3/50
234366/234366 [==============================] - 123s 525us/step - loss: 7.1154 - accuracy: 0.0464

Epoch 00003: loss improved from 7.15572 to 7.11541, saving model to ./model_weights.hdf5
Epoch 4/50
234366/234366 [==============================] - 124s 528us/step - loss: 7.0725 - accuracy: 0.0463

Epoch 00004: loss improved from 7.11541 to 7.07245, saving model to ./model_weights.hdf5
Epoch 5/50
234366/234366 [==============================] - 124s 530us/step - loss: 7.0271 - accuracy: 0.0466

Epoch 00005: loss improved from 7.07245 to 7.02707, saving model to ./model_weights.hdf5
Epoc

Epoch 42/50
234366/234366 [==============================] - 126s 538us/step - loss: 4.7807 - accuracy: 0.1222

Epoch 00042: loss improved from 4.81258 to 4.78065, saving model to ./model_weights.hdf5
Epoch 43/50
234366/234366 [==============================] - 126s 540us/step - loss: 4.7446 - accuracy: 0.1265

Epoch 00043: loss improved from 4.78065 to 4.74456, saving model to ./model_weights.hdf5
Epoch 44/50
234366/234366 [==============================] - 130s 556us/step - loss: 4.7132 - accuracy: 0.1286

Epoch 00044: loss improved from 4.74456 to 4.71319, saving model to ./model_weights.hdf5
Epoch 45/50
234366/234366 [==============================] - 123s 526us/step - loss: 4.6794 - accuracy: 0.1313

Epoch 00045: loss improved from 4.71319 to 4.67939, saving model to ./model_weights.hdf5
Epoch 46/50
234366/234366 [==============================] - 124s 528us/step - loss: 4.6418 - accuracy: 0.1360

Epoch 00046: loss improved from 4.67939 to 4.64176, saving model to ./model_weights.

In [120]:
from keras.preprocessing.sequence import pad_sequences

import numpy as np


def gen(model,seq,max_len = 15):
    ''' Generates a sequence given a string seq using specified model until the total sequence length
    reaches max_len'''
    # Tokenize the input string
    tokenized_sent = tokenizer.texts_to_sequences([seq])
#     max_len = max_len+len(tokenized_sent[0])

    # If sentence is not as long as the desired sentence length, we need to 'pad sequence' so that
    # the array input shape is correct going into our LSTM. the `pad_sequences` function adds 
    # zeroes to the left side of our sequence until it becomes 19 long, the number of input features.
    gen_res = seq.split(' ')
    
#     while len(tokenized_sent[0]) < max_len:
#         padded_sentence = pad_sequences(tokenized_sent[-10:],maxlen=10)
#         op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
#         tokenized_sent[0].append(op.argmax()+1)
        
    while len(gen_res) < max_len:
        padded_sentence = pad_sequences(tokenized_sent[-10:],maxlen=10)
        op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
        tokenized_sent[0].append(op.argmax()+1)
        gen_res.append(reverse_word_map[op.argmax()+1])
        
#     return " ".join(map(lambda x : reverse_word_map[x],tokenized_sent[0]))
    return " ".join(gen_res)

In [121]:
print(gen(model, 'Сценарий фильма явно состряпан на основе тех штампов, которые обеспечивают'))

Сценарий фильма явно состряпан на основе тех штампов, которые обеспечивают пытаются его диалогов и премии


In [122]:
my_test_sentences = [
    'Сценарий фильма явно состряпан на основе тех штампов, которые обеспечивают',
    'Автору высшую степень рукопожатности и признательность тех, для кого снимали',
    'Фильм - откровенный лубок со всеми штампами антисоветской и русофобской пропаганды',
    'В общем, фильм ничего кроме омерзения не вызывает, настолько топорно воспроизводится',
    'Прямо скажем, фильм по своему гениален. Потому что это ж',
    'Начну несколько издали, потому что в упор писать о сериале',
    'Сериал "Зулейха открывает глаза" это очередная попытка убедить зрителя, что',
    'За исключением ряда преувеличений, анахронизмов и стереотипов, допустимых для художественного',
    'Как итог, фильм получился легким, светлым, с хорошим чувством юмора',
    'Серии несколько высосаны из пальца, при ламповой и уникальной атмосфере',
    'У Смешариков есть все шансы снова стать чем-то культовым и',
    'Без лишней скромности и без преувеличения стоит отметить что это',
    'Здесь есть основной макросюжет и масса мелких микросюжетных зарисовок, которые',
    'Можете смотреть этот мульт как развлечение, не вдаваясь в',
    'Резюмируя, отметим, что сериал безукоризненно работает на всех уровнях – техническом,',
    'Это увлекательное шоу с различными персонажами, научной тематикой и совершенствованием',
    'Крайне качественно сделанный сериал с крепким сюжетом и качественным подбором',
    'Множество сюжетных линий расчерчены как по линейке; и смотреть отдельно',
    'Думаю это не столь значительно, ведь можно просто наслаждаться всплеском',
    'Уверен, что мы с вами еще тысячу раз удивимся и'  
]

In [123]:
for i, value in enumerate(my_test_sentences, 1):
    print(i, gen(model, value))

1 Сценарий фильма явно состряпан на основе тех штампов, которые обеспечивают пытаются его диалогов и премии
2 Автору высшую степень рукопожатности и признательность тех, для кого снимали то 10 сказать хотелось простительно
3 Фильм - откровенный лубок со всеми штампами антисоветской и русофобской пропаганды сценаристы а и бесконечное
4 В общем, фильм ничего кроме омерзения не вызывает, настолько топорно воспроизводится что не и и
5 Прямо скажем, фильм по своему гениален. Потому что это ж и наблюдали конечно и образом
6 Начну несколько издали, потому что в упор писать о сериале связано несколько да так что
7 Сериал "Зулейха открывает глаза" это очередная попытка убедить зрителя, что и за — что в
8 За исключением ряда преувеличений, анахронизмов и стереотипов, допустимых для художественного единственные и в фильме и
9 Как итог, фильм получился легким, светлым, с хорошим чувством юмора не считаю человек будут концепций
10 Серии несколько высосаны из пальца, при ламповой и уникальной атмосф